<a href="https://colab.research.google.com/github/NeuromatchAcademy/course-content-dl/blob/w2d3_t1_initial/tutorials/W2D3_ModernRecurrentNeuralNetworks/W2D3_Tutorial1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Neuromatch Academy: Week 2, Day 3, Tutorial 1
# Modern RNNs

__Content creators:__ James Evans, Bhargav Srinivasa Desikan, Anis Zahedifard

__Content reviewers:__ XXXXX, XXXXXX

__Content editors:__ XXXXX

__Production editors:__ Roberto Guidotti, Spiros Chavlis, XXXXXX




In [1]:
# This is a place holder notebook. DO NOT USE.

import numpy as np

In [2]:
for i in range(10):
  print(f'Power of 2: {i**2}')

Power of 2: 0
Power of 2: 1
Power of 2: 4
Power of 2: 9
Power of 2: 16
Power of 2: 25
Power of 2: 36
Power of 2: 49
Power of 2: 64
Power of 2: 81


In [3]:
def myfunc(counter):
  for i in range(counter):
    raise NotImplementedError
    j = ...
    print(f'Power of 2: {j}')


## Uncomment to check your function
# myfunc(15)

In [4]:
#to_remove solution
def myfunc(counter):
  for i in range(counter):
    j = i ** 3
    print(f'Power of 2: {j}')


myfunc(15)

Power of 2: 0
Power of 2: 1
Power of 2: 8
Power of 2: 27
Power of 2: 64
Power of 2: 125
Power of 2: 216
Power of 2: 343
Power of 2: 512
Power of 2: 729
Power of 2: 1000
Power of 2: 1331
Power of 2: 1728
Power of 2: 2197
Power of 2: 2744
